In [1]:
import sys
import os
IMATOOLS_DIR = os.getcwd()+'/..'
sys.path.insert(1, IMATOOLS_DIR) 


from imatools.common import ioutils as iou
from imatools.common import vtktools as vtku

import pandas as pd 
import numpy as np


In [2]:

# Locations of hard drive based on platform
dirdic={'macOS'   : '/Volumes/sandisk', 
        'Linux'   : '/media/jsl19/sandisk' , 
        'Windows' : 'D:/'}

p2f = iou.fullfile(dirdic[iou.chooseplatform()], '09-dnav_vs_inav/umc')
name = 'spatial_correspondence.csv'

df = pd.read_csv(iou.fullfile(p2f, 'local',name), skipinitialspace=True)
df_stats = pd.read_csv(iou.fullfile(p2f, 'results_local', 'stats.csv'), skipinitialspace=True)

In [3]:
def get_data_from_vtk_path(path, type='cell') : 
    msh = vtku.readVtk(path)
    if type=='cell' : 
        dat = vtku.convertCellDataToNpArray(msh, 'scalars')
    else : # type=='point'
        dat = vtku.convertPointDataToNpArray(msh, 'scalars')
    return dat 

CX = ['2', '5', '6', '8', '9', '11', '12', '13', '15', '16', '17', '18', '19', '21', '22', '24', '26', '7', '10', '14', '20', '23', '25']

xnav = lambda x : 'LGE_{}NAV'.format(x)
dname = 'Normalised_IIR_MaxScar-single-voxel.vtk'
iname = 'Scalars_i_on_d.vtk'


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as sstat

# cx = 0
fig, ax = plt.subplots(4, 6, sharex=True, sharey=True,squeeze=True)
fig.set_figwidth(18)
fig.set_figheight(12)

sns.set_palette('colorblind')

corrs = np.zeros(len(CX))

for cx in range(len(CX)) : 
    ddat = get_data_from_vtk_path(iou.fullfile(p2f, 'local', CX[cx], xnav('d'), 'OUTPUT', dname))
    idat = get_data_from_vtk_path(iou.fullfile(p2f, 'local', CX[cx], iname))

    df_test = pd.DataFrame({
        'scalars': ddat.tolist() + idat.tolist(), 
        'nav': ['dNAV'] * len(ddat) + ['iNAV']*len(idat)
        })

    subs = np.unravel_index(cx, (4,6))
    corrs[cx] = sstat.pearsonr(ddat, idat).statistic

    ax[subs].set_title('Case {} (r={:.2f})'.format(CX[cx], corrs[cx]))
    sns.histplot(ax=ax[subs], data=df_test, x='scalars', hue='nav', 
                element='step', bins=50, multiple='dodge', stat='count', 
                binrange=(0, 2.5))


# plt.savefig(iou.fullfile(iou.fullfile(p2f, 'local', 'scalar_comparison.eps')))



In [ ]:
case_mean_diff = np.zeros(len(CX))
for cx in range(len(CX)):
    ddat = get_data_from_vtk_path(iou.fullfile(p2f, 'local', CX[cx], xnav('d'), 'OUTPUT', dname))
    idat = get_data_from_vtk_path(iou.fullfile(p2f, 'local', CX[cx], iname))

    case_mean_diff[cx] = np.mean(np.subtract(ddat, idat))


sns.boxplot(y=case_mean_diff)
print(sstat.ttest_1samp(a=case_mean_diff, popmean=0))

In [ ]:
# import scipy.stats as sstat
# help(sstat.ttest_1samp)

def print_stats(arr) :
    print('{} +/- {}'.format(np.mean(arr), np.std(arr)))

print_stats(case_mean_diff)
print_stats(corrs)
print(iou.get_boxplot_values(corrs))
# print('{} +/- {}'.format(np.mean(case_mean_diff), np.std(case_mean_diff)))
# print('{} +/- {}'.format(np.mean(case_mean_diff), np.std(case_mean_diff)))
print('')
# sns.histplot(x=case_mean_diff, bins=20)

In [ ]:
# 
#print('{:.2f} +/- {:.2f}'.format(np.nanmean(corrs),np.nanstd(corrs)))

np.sort(CX)

In [ ]:
# import matplotlib.pyplot as plt

# df_stats['LGE_TYPE']=='iNAV'], x='CASE', y='MEAN_BP', yerr='SDEV_BP', fmt='or')
# else :
#     sns.scatterplot(data=df_stats, x='CASE', y='FIB_SCORE ', hue='LGE_TYPE')

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

x=np.linspace(0, 0.7, 10)
y=x

sns.set_palette(palette='Pastel1')
sns.scatterplot(data=df, x='fibrosis_d', y='fibrosis_i', hue='th_inav', hue_order=[1.1205, 1.13, 1.1367, 1.1504, 1.2, 1.265])
plt.plot(x,y)


In [ ]:
fig, ax = plt.subplots(1,2)
fig.set_figheight(5)
fig.set_figwidth(15)
sns.set_palette('Pastel1')

sns.boxplot(ax=ax[0], x=df['th_inav'], y=(df['fibrosis_d']-df['fibrosis_i'])
            ** 2, order=[1.1205, 1.13, 1.1367, 1.1504, 1.2, 1.265], 
            whis=1)
sns.scatterplot(ax=ax[1], data=df, x='fibrosis_d', y='fibrosis_i', hue='th_inav', hue_order=[1.1205, 1.13, 1.1367, 1.1504, 1.2, 1.265])
plt.plot()

ax[0].set_xlabel('iNAV IIR Threshold')
ax[0].set_ylabel('Mean Squared Error')
ax[1].set_xlabel('dNAV Fibrosis %')
ax[1].set_ylabel('iNAV Fibrosis %')

# sns.lmplot( data=df, x='fibrosis_d', y='fibrosis_i', col='th_inav', col_order=[1.1205, 1.13, 1.1367, 1.1504, 1.2, 1.265])

In [ ]:

fig, ax = plt.subplots(1)
fig.set_figheight(5)
fig.set_figwidth(15)
sns.set_palette('Pastel1')
sns.boxplot(ax=ax, data=df, 
    x='perf_name', y='performance', hue='th_inav', whis=1,
    hue_order=[1.1205, 1.13, 1.1367, 1.1504, 1.2, 1.265])

plt.savefig(iou.fullfile(p2f, 'performance.eps'))


In [ ]:
metric_name = 'dice'
th = [1.1205, 1.13, 1.1367, 1.1504, 1.2, 1.265]

print(metric_name)
for threshold in th :
    # threshold = 1.1205
    metric = df['performance']
    thresh = df['th_inav']

    metric = metric[df['perf_name']==metric_name]
    thresh = thresh[df['perf_name']==metric_name]

    metric = metric[thresh==threshold]

    print('{:.3f} (+/-{:.3f})'.format(np.mean(metric), np.std(metric)))


# Point analysis 

+ Load a case 
+ Move cell data to point data 
+ Explore

**Sanity check**: check that the correct data are being retrieved - _PASSED_

Compare the point data coming out of the mesh to the point data after converting cell to point.

In [ ]:
cx = 2

dpath = iou.fullfile(p2f, 'local', CX[cx], xnav('d'), 'OUTPUT', dname)
ipath = iou.fullfile(p2f, 'local', CX[cx], iname) 

dmsh = vtku.readVtk(dpath)
imsh = vtku.readVtk(ipath)
ddat = vtku.convertPointDataToNpArray(dmsh, 'scalars')
idat = vtku.convertPointDataToNpArray(imsh, 'scalars')

dmsh_pts = vtku.set_cell_to_point_data(dmsh, 'scalars')
imsh_pts = vtku.set_cell_to_point_data(imsh, 'scalars')
ddat_pts = vtku.convertPointDataToNpArray(dmsh_pts, 'scalars')
idat_pts = vtku.convertPointDataToNpArray(imsh_pts, 'scalars')

print('len idat: {}, len idat_pts: {}'.format(len(idat), len(idat_pts)))
print('idat - idat_pts : {}'.format(np.mean(np.subtract(idat, idat_pts))))
# ddat = get_data_from_vtk_path(dpath)
# idat = get_data_from_vtk_path(ipath)


Create all point-based `scalars_i_on_d` files. 

> NOTE: dNAV normalised scalars already have the point data saved in them.

In [ ]:
# dpath = iou.fullfile(p2f, 'local', CX[cx], xnav('d'), 'OUTPUT', dname)
# cx = 2
def fileparts(pth):
    directory = os.path.dirname(pth)
    fname_ext = os.path.basename(pth)
    aux = os.path.splitext(fname_ext)
    fname = aux[0]
    ext = aux[1]

    return directory, fname, ext


for cx in range(len(CX)) : 
    
    ipath = iou.fullfile(p2f, 'local', CX[cx], iname)
    imsh = vtku.readVtk(ipath)
    imsh_pts = vtku.set_cell_to_point_data(imsh, 'scalars')

    di, fn, xt = fileparts(ipath)
    iname_pts = fn + "_pts"
    vtku.writeVtk(imsh_pts, di, iname_pts)


+ We ran the `10_project_inav_on_dnav.sh` script, commenting the projection (as we do not need to create the `scalars_i_on_d` files). 
+ Once done, we rerun some of the calculations done before
+ see below

In [4]:
p2f = iou.fullfile(dirdic[iou.chooseplatform()], '09-dnav_vs_inav/umc')
name = 'spatial_correspondence_pts.csv'

df_pts = pd.read_csv(iou.fullfile(p2f, 'local', name), skipinitialspace=True)

df_pts


,th_dnav,th_inav,fibrosis_d,fibrosis_i,performance,perf_name
0,1.2,1.2000,0.242276,0.170156,0.229543,jaccard
1,1.2,1.2000,0.242276,0.170156,0.442933,precision
2,1.2,1.2000,0.242276,0.170156,0.322706,recall
3,1.2,1.2000,0.242276,0.170156,0.736644,accuracy
4,1.2,1.2000,0.242276,0.170156,0.373380,dice
...,...,...,...,...,...,...
685,1.2,1.1367,0.639512,0.580759,0.644086,jaccard
686,1.2,1.1367,0.639512,0.580759,0.812426,precision
687,1.2,1.1367,0.639512,0.580759,0.756597,recall
688,1.2,1.1367,0.639512,0.580759,0.731082,accuracy


In [ ]:

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

x=np.linspace(0, 0.7, 10)
y=x

sns.set_palette(palette='Pastel1')
sns.scatterplot(data=df_pts, x='fibrosis_d', y='fibrosis_i', hue='th_inav', hue_order=[1.1205, 1.13, 1.1367, 1.1504, 1.2, 1.265])
plt.plot(x,y)

In [ ]:
fig, ax = plt.subplots(1)
fig.set_figheight(5)
fig.set_figwidth(15)
sns.set_palette('Pastel1')
sns.boxplot(ax=ax, data=df_pts,
            x='perf_name', y='performance', hue='th_inav', whis=1,
            hue_order=[1.1205, 1.13, 1.1367, 1.1504, 1.2, 1.265])

# plt.savefig(iou.fullfile(p2f, 'performance.eps'))


In [ ]:
metric_name = 'dice'
th = [1.1205,1.13, 1.1367, 1.1504, 1.2, 1.265]

print(metric_name)
for threshold in th:
    # threshold = 1.1205
    metric = df_pts['performance']
    thresh = df_pts['th_inav']

    metric = metric[df_pts['perf_name'] == metric_name]
    thresh = thresh[df_pts['perf_name'] == metric_name]

    metric = metric[thresh == threshold]

    print('{:.3f} (+/-{:.3f})'.format(np.mean(metric), np.std(metric)))


# Sweep thresholds - get fibrosis scores 

First get the point fibrosis scores from all dnavs

In [5]:
import numpy as np
dname = 'Normalised_IIR_MaxScar-single-voxel.vtk'
iname_pts = 'Scalars_i_on_d_pts.vtk'

sweep = np.arange(0.7, 1.59, 0.01)
len_sweep = len(sweep)

args_min = np.zeros((len(sweep), 1), dtype=int)

dfib_scores = np.zeros((len(CX), 1))
ifib_best_scores = np.zeros_like(dfib_scores)
ifib_best_thresholds = np.zeros_like(dfib_scores)
ifib_scores_1d2 = np.zeros_like(ifib_best_scores)

sq_err_1d2 = np.zeros_like(dfib_scores)
sq_err_best = np.zeros_like(dfib_scores)
for cx in range(len(CX)):
    dpath = iou.fullfile(p2f, 'local', CX[cx], xnav('d'), 'OUTPUT', dname)
    dmsh = vtku.readVtk(dpath)

    dfib_scores[cx] = vtku.fibrosis_score_point(dmsh, 1.2)

    ipath_pts = iou.fullfile(p2f, 'local', CX[cx], iname_pts)
    imsh_pts = vtku.readVtk(ipath_pts)

    sweep_scores = np.zeros((len_sweep, 1))
    for ix in range(len_sweep):
        sweep_scores[ix] = vtku.fibrosis_score_point(imsh_pts, sweep[ix])

    sq_err = np.power(sweep_scores-dfib_scores[cx], 2)
    args_min[cx] = np.argmin(sq_err)
    
    ifib_scores_1d2[cx] = sweep_scores[50] 
    sq_err_1d2[cx] = sq_err[50]

    ifib_best_thresholds[cx] = sweep[args_min[cx]]
    ifib_best_scores[cx] = sweep_scores[args_min[cx]]
    sq_err_best[cx] = sq_err[args_min[cx]]


Wilcoxon test on new fibrosis values

In [13]:
import scipy.stats as sstat
print('Ranksum')
print(sstat.ranksums(x=dfib_scores, y=ifib_scores_1d2))
print("Ttest")
print(sstat.ttest_1samp(a=dfib_scores-ifib_scores_1d2, popmean=0))
print("Like before, we reject the null hypothesis that the fibrosis scores are equal")


Ranksum
RanksumsResult(statistic=array([2.22987183]), pvalue=array([0.02575595]))
Ttest
Ttest_1sampResult(statistic=array([4.08791504]), pvalue=array([0.00048674]))
Like before, we reject the null hypothesis that the fibrosis scores are equal


## Best thresholds: 

In [16]:
print('mean = {}'.format(np.mean(ifib_best_thresholds)))
print('median = {}'.format(np.median(ifib_best_thresholds)))

print(iou.get_boxplot_values(ifib_best_thresholds))

mean = 1.1500000000000004
median = 1.1400000000000003
{'min': 1.0300000000000002, 'low_whisker': 1.0300000000000002, 'low_quartile': 1.1000000000000003, 'median': 1.1400000000000003, 'high_quartile': 1.1800000000000004, 'high_whisker': 1.3000000000000005, 'max': 1.3500000000000005}


## Refined search

Sweep range = `1.05:0.0001:1.35`

Mean threshold = 1.15
Median threshold = 1.14 

In [26]:
dname = 'Normalised_IIR_MaxScar-single-voxel.vtk'
iname_pts = 'Scalars_i_on_d_pts.vtk'

rsweep = np.arange(1.03, 1.35, 0.001)
len_rsweep = len(rsweep)

args_min = np.zeros((len(CX), 1), dtype=int)

ifib_rsweep_scores = np.zeros((len(CX), len_rsweep))
print(len(rsweep))
for cx in range(len(CX)):
    print(cx)
    ipath_pts = iou.fullfile(p2f, 'local', CX[cx], iname_pts)
    imsh_pts = vtku.readVtk(ipath_pts)

    rsweep_scores = np.zeros((len_rsweep, 1))
    for ix in range(len_rsweep):
        ifib_rsweep_scores[cx, ix] = vtku.fibrosis_score_point(imsh_pts, rsweep[ix])

    args_min[cx] = np.argmin(np.power(ifib_rsweep_scores[cx, :] - dfib_scores[cx], 2))



321
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22


## specialised threshold

+ Get a random permutation of 18 cases for 'training', the remaining 5 will be for testing
+ Find which threshold minimises across all cases (sum columns and pick minimum)
+ Do this 100 times with different permutations
+ Keep the 3 thresholds that perform best.

In [33]:
np.power(ifib_rsweep_scores-dfib_scores,2)

array([[1.67628982e-01, 1.66534722e-01, 1.65394554e-01, ...,
        8.80975323e-03, 9.00498522e-03, 9.08599676e-03],
       [1.17867755e-01, 1.15261569e-01, 1.14278858e-01, ...,
        4.73818089e-02, 4.77147177e-02, 4.77814394e-02],
       [8.04239062e-02, 7.97090748e-02, 7.97090748e-02, ...,
        5.25515280e-02, 5.25778551e-02, 5.27359564e-02],
       ...,
       [1.74371805e-01, 1.71485341e-01, 1.71166110e-01, ...,
        1.61289050e-02, 1.63451107e-02, 1.64240879e-02],
       [3.17193867e-05, 6.34914183e-05, 1.33797560e-04, ...,
        2.48814363e-01, 2.49381590e-01, 2.50012600e-01],
       [1.66194280e-02, 1.58606713e-02, 1.48489178e-02, ...,
        1.61026429e-01, 1.61094705e-01, 1.61436300e-01]])